# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [6]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [7]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_2 (Dense)             (None, 100)               100100    
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


## Orgainze test and training data
Be sure to change file contents

In [8]:
trainingValues = [0] * 691 + [1] * 700
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
# testImages = keras.utils.image_dataset_from_directory(
#     directory='testData',
#     labels= testValues,
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))
# print (trainImages)

Found 1391 files belonging to 2 classes.


## Prediction before training

In [9]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-11-05 09:50:19.495375: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-05 09:50:19.588438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
[[0.5159187]]


## Train Last two layers

In [10]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-11-05 09:50:21.145189: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 17s 322ms/step - loss: 0.6594 - accuracy: 0.8174
Epoch 2/10
44/44 [==============================] - 14s 312ms/step - loss: 0.5546 - accuracy: 0.8850
Epoch 3/10
44/44 [==============================] - 14s 311ms/step - loss: 0.4280 - accuracy: 0.9109
Epoch 4/10
44/44 [==============================] - 14s 312ms/step - loss: 0.3310 - accuracy: 0.9166
Epoch 5/10
44/44 [==============================] - 14s 312ms/step - loss: 0.2695 - accuracy: 0.9274
Epoch 6/10
44/44 [==============================] - 14s 312ms/step - loss: 0.2302 - accuracy: 0.9303
Epoch 7/10
44/44 [==============================] - 14s 312ms/step - loss: 0.2035 - accuracy: 0.9403
Epoch 8/10
44/44 [==============================] - 14s 312ms/step - loss: 0.1836 - accuracy: 0.9389
Epoch 9/10
44/44 [==============================] - 14s 312ms/step - loss: 0.1700 - accuracy: 0.9432
Epoch 10/10
44/44 [==============================] - 14s 311ms/step - loss: 0.1580 - accuracy: 0.9468


## Test Some data

In [11]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [12]:
from PIL import Image
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 21ms/step
[[0.00419484]]


## Save the model for export

In [13]:
completeModel.save("weaselModel.keras")

## Check test data for each animal

In [14]:
# import required module
import os
# assign directory
directory = 'extraTest/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

extraTest/testDeer/1705.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.02239864]]
extraTest/testDeer/1687.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 24ms/step
[[0.01060795]]
extraTest/testDeer/1703.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.06498446]]
extraTest/testDeer/1692.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.03798332]]
extraTest/testDeer/1673.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00835038]]
extraTest/testDeer/1694.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01026658]]
extraTest/testDeer/1681.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00490242]]
extraTest/testDeer/1679.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00605117]]
extraTest/testDeer/1698.

In [15]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/3282.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 31ms/step
[[0.02400717]]
extraTest/testFox/3295.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.15512948]]
extraTest/testFox/3278.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01211269]]
extraTest/testFox/3294.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.16379616]]
extraTest/testFox/3287.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6122542]]
extraTest/testFox/3297.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.18269859]]
extraTest/testFox/3281.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.12155272]]
extraTest/testFox/3292.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step


In [16]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02205982]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.05376914]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.11721212]]
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.03867833]]
extraTest/testPossum/1995.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.17280129]]
extraTest/testPossum/1986.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00337602]]
extraTest/testPossum/1974.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00862576]]
extraTest/testPossum/1967.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01426808]]
extraTes

In [17]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.1294704]]
extraTest/testRabbit/765.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.13496095]]
extraTest/testRabbit/776.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.2298127]]
extraTest/testRabbit/770.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.10562854]]
extraTest/testRabbit/763.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.11995129]]
extraTest/testRabbit/758.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.28328905]]
extraTest/testRabbit/741.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01725728]]
extraTest/testRabbit/752.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.08174185]]
extraTest/testRabb

In [18]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.70874333]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.6060391]]
extraTest/testSquirrel/118.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.78555727]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.7274156]]
extraTest/testSquirrel/125.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.92327327]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9186049]]
extraTest/testSquirrel/122.jpg___crop03_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.3115012]]
extraTest/testSquirrel/101.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.97456896]]
extraTes

In [19]:
directory = 'extraTest/testDog'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testDog/5654.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.5521695]]
extraTest/testDog/5645.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.33403006]]
extraTest/testDog/5657.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00428138]]
extraTest/testDog/5648.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7757876]]
extraTest/testDog/5651.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.05399832]]
extraTest/testDog/5649.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.04076986]]
extraTest/testDog/5656.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05221171]]
extraTest/testDog/5655.

In [20]:
directory = 'extraTest/testCat'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.06885132]]
extraTest/testCat/7043.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.02489963]]
extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.08391421]]
extraTest/testCat/7044.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9636802]]
extraTest/testCat/7045.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9885671]]
extraTest/testCat/7048.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99485296]]
extraTest/testCat/7045.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99198323]]
extraTest/testCat/7047.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [

In [21]:
directory = 'extraTest/testCoyote'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCoyote/58ac0d93-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01257941]]
extraTest/testCoyote/58b6643a-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.02588589]]
extraTest/testCoyote/5a049387-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.04555858]]
extraTest/testCoyote/58adc235-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.7976709]]
extraTest/testCoyote/58ac0d6c-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05762282]]
extraTest/testCoyote/5a096aec-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01823562]]
extraTest/testCoyote/5a049367-23d2-11e8-a6a3-ec086b02610b.jpg___c

In [23]:
directory = 'extraTest/testWeasel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testWeasel/image_01408.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.08638817]]
extraTest/testWeasel/image_01071.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.8906476]]
extraTest/testWeasel/image_01072.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9364103]]
extraTest/testWeasel/image_00821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.86800826]]
extraTest/testWeasel/image_00821.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8230953]]
extraTest/testWeasel/image_01408.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.2659571]]
extraTest/testWeasel/image_01071.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 17ms/step
[[0.85238385]]
extraTest/testWeasel/image_00622.jpg___

## Code for data manipulation to creaet more data

In [22]:
# import os
# from PIL import Image
# directory = 'extraTraining/cropped_weasel' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])